In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

# Define your linear evaluation model architecture
class LinearEvaluationModel(nn.Module):
    def __init__(self, num_classes):
        super(LinearEvaluationModel, self).__init__()
        # Define your linear evaluation model layers here
        self.fc = nn.Linear(18 * 10, num_classes)

    def forward(self, x):
        # Define the forward pass of your linear evaluation model here
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Load the learned representations from the self-supervised model
representations = torch.load("../input/representations/representations.pth")  # Load the representations tensor

# Set random seed for reproducibility
torch.manual_seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters for linear evaluation
num_epochs = 10
batch_size = 18  # Adjust the batch size to match the representations_batch size
learning_rate = 0.001
gamma = 0.1  # Learning rate decay factor
step_size = 5  # Number of epochs after which to decay learning rate

# Define your data transformations for linear evaluation
transform = transforms.Compose([
    transforms.Resize((100, 100)),  # Resize input images to match the representation size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image tensors
])

# Set the path to your labeled dataset for facial expression recognition
labeled_dataset_dir = "../input/face-expression-recognition-dataset/"

# Create the labeled dataset
labeled_dataset = ImageFolder(labeled_dataset_dir, transform=transform)

# Adjust the batch size of the labeled dataset to match the representations batch size
labeled_dataloader = DataLoader(labeled_dataset, batch_size=batch_size, shuffle=True)

# Create the linear evaluation model
linear_model = LinearEvaluationModel(num_classes=len(labeled_dataset.classes)).to(device)

# Define your loss function for linear evaluation
criterion = nn.CrossEntropyLoss()

# Define your optimizer for linear evaluation
optimizer = optim.Adam(linear_model.parameters(), lr=learning_rate)

# Define your learning rate scheduler for linear evaluation
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

# Training loop for linear evaluation
for epoch in range(num_epochs):
    linear_model.train()  # Set the linear evaluation model to training mode
    for batch_idx, (images, labels) in enumerate(labeled_dataloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        representations_batch = representations[batch_idx * batch_size : (batch_idx + 1) * batch_size].to(device)
        outputs = linear_model(representations_batch)

        # Compute the linear evaluation loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss after every few iterations
        if (batch_idx+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(labeled_dataloader)}], Loss: {loss.item():.4f}')

    # Update the learning rate scheduler
    scheduler.step()

    # Display epoch information
    print(f"End of Epoch {epoch+1}. Learning Rate: {scheduler.get_last_lr()[0]}")

# End of linear evaluation
print("Linear evaluation finished.")

# You can now use the linear_model for facial expression recognition on new data
# Evaluate the accuracy on a separate test set or perform predictions on new images


RuntimeError: mat1 and mat2 shapes cannot be multiplied (18x10 and 180x1)